# 02 - Analysis
* Load the depth data created during preprocessing
* Load weights from the provided PFRA JSON file
* Estimate depth, upper CI, and lower CI for each pixel for each return period

In [1]:
import ffrd_data_utils as fdu
import xarray as xr
import json

In [2]:
from dask.distributed import Client

client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 15.48 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38033,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.48 GiB
Comm: tcp://127.0.0.1:46053,Total threads: 3
Dashboard: http://127.0.0.1:44159/status,Memory: 3.87 GiB
Nanny: tcp://127.0.0.1:34501,


In [3]:
DEPTH_ZARR = "./data/depth.zarr"
WEIGHTS = "./data/Addison_M03_H04_Weights.json"
RETURN_PERIODS = [10, 50, 100, 500]
AEPS = [1/return_period for return_period in RETURN_PERIODS]
WSEL_RECHUNKED_ZARR = "./data/wsel_rechunked.zarr"

In [4]:
# load the pre-processed depth dataset from the previous notebook
depth_ds = xr.open_zarr(DEPTH_ZARR)
depth_ds

<xarray.Dataset>
Dimensions:      (r: 96, y: 1024, x: 1024)
Coordinates:
  * r            (r) int64 0 1 2 3 4 5 6 7 8 9 ... 86 87 88 89 90 91 92 93 94 95
  * x            (x) float64 1.091e+06 1.091e+06 ... 1.096e+06 1.096e+06
  * y            (y) float64 1.919e+06 1.919e+06 ... 1.914e+06 1.914e+06
Data variables:
    depth        (r, y, x) float32 dask.array<chunksize=(96, 16, 16), meta=np.ndarray>
    spatial_ref  int64 ...

In [5]:
# load weights for each depth raster
with open(WEIGHTS, 'r') as weights_file:
    weights = list(json.load(weights_file).values())

In [6]:
# for each return period:
# 1. estimate the depth at each pixel
# 2. compute the upper and lower CI for the depth at each pixel
depth_quantiles_ds = fdu.depth_quantiles(depth_ds.depth, AEPS, spatial_ref=depth_ds.spatial_ref, weights=weights)
depth_quantiles_ds

<xarray.Dataset>
Dimensions:      (aep: 4, y: 1024, x: 1024)
Coordinates:
  * y            (y) float64 1.919e+06 1.919e+06 ... 1.914e+06 1.914e+06
  * x            (x) float64 1.091e+06 1.091e+06 ... 1.096e+06 1.096e+06
  * aep          (aep) float64 0.1 0.02 0.01 0.002
Data variables:
    depth        (aep, y, x) float32 dask.array<chunksize=(4, 16, 16), meta=np.ndarray>
    lower_ci     (aep, y, x) float32 dask.array<chunksize=(4, 16, 16), meta=np.ndarray>
    upper_ci     (aep, y, x) float32 dask.array<chunksize=(4, 16, 16), meta=np.ndarray>
    spatial_ref  int64 ...

In [7]:
%%time
# write results to disk
depth_quantiles_ds.to_zarr('./data/depth_quantiles.zarr', mode='w')

/home/thomaswilliams3/miniconda3/envs/ffrd-grid-concept/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/thomaswilliams3/miniconda3/envs/ffrd-grid-concept/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/home/thomaswilliams3/miniconda3/envs/ffrd-grid-concept/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/thomaswilliams3/miniconda3/envs/ffrd-grid-concept/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/home/thomaswilliams3/miniconda3/envs/ffrd-grid-concept/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarni

CPU times: user 40min 1s, sys: 6min 43s, total: 46min 44s
Wall time: 2h 51min 26s


In [8]:
# check out the results
depth_quantiles_ds = xr.open_zarr('./data/depth_quantiles.zarr')
depth_quantiles_ds

<xarray.Dataset>
Dimensions:      (aep: 4, y: 1024, x: 1024)
Coordinates:
  * aep          (aep) float64 0.1 0.02 0.01 0.002
  * x            (x) float64 1.091e+06 1.091e+06 ... 1.096e+06 1.096e+06
  * y            (y) float64 1.919e+06 1.919e+06 ... 1.914e+06 1.914e+06
Data variables:
    depth        (aep, y, x) float32 dask.array<chunksize=(4, 16, 16), meta=np.ndarray>
    lower_ci     (aep, y, x) float32 dask.array<chunksize=(4, 16, 16), meta=np.ndarray>
    spatial_ref  int64 ...
    upper_ci     (aep, y, x) float32 dask.array<chunksize=(4, 16, 16), meta=np.ndarray>